In [1]:
import numpy as np
from qiskit import *                 
from qiskit_aer import *             
from qiskit_ibm_runtime import *

In [2]:
# Variables
machine = 'simulator'            # Chosen machine to submit jobs to
num_qubits = 30                  # Number of qubits to run on
num_shots = 1024                 # Number of shots to take
chunk_size = 30                  # Size of the chunking for the mod2 and iteration methods
mod2_mods = 3                    # Number of times to apply mod2. The value inputted results in 2 runs per 1 value. mod2_mods=3 --> 6 jobs QPU submitted

In [3]:
# Lets us use IBM machines
QiskitRuntimeService.save_account(channel = 'ibm_quantum', token = '4b3a2ecb92b9446de636396b89dd76f0d61c2715122c6a353720e5b58c124a7d3d20528012332d57add9a37a61aaf508b54c97698ddc770f922d68fd213b1b25',
                                  overwrite = True, set_as_default = True)
service = QiskitRuntimeService(instance = "ibm-q/open/main")

RequestsApiError: 'HTTPSConnectionPool(host=\'auth.quantum-computing.ibm.com\', port=443): Max retries exceeded with url: /api/version (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x14511e120>: Failed to resolve \'auth.quantum-computing.ibm.com\' ([Errno 8] nodename nor servname provided, or not known)"))'

In [ ]:
# Increased QRNG on Aer Simulator
def number_generator_simulator(num_qubits, num_shots):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    simulator = AerSimulator()                                    # Lets us use the Aer Simulator 
    compiled_circuit = transpile(circ, simulator)                 # Compiled circuit using Aer 
    result = simulator.run(compiled_circuit, shots=num_shots, memory=True).result()      # Result with the 10000 shots as to not run forever on IBM machines
    raw_data = result.get_memory()
    data = ''.join(raw_data)

    return data     # Returns a concatenated string of all binary digits in the order that they were measured

In [ ]:
# Increased QRNG on ibm_brisbane
def number_generator_brisbane(num_qubits, num_shots):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    # Runs the QRNG on ibm_brisbane with num_shots
    brisbane_backend = service.backend('ibm_brisbane')            # Creates a backend with ibm_brisbane
    pm = generate_preset_pass_manager(backend=brisbane_backend, optimization_level=3)   # Can change optimization_level to whatever's desired
    isa_circuit = pm.run(circ)

    with Session(backend=brisbane_backend) as session:
        sampler = Sampler(mode=session)
        job = sampler.run([isa_circuit], shots=num_shots)
        counts = job.result()[0].data.c.get_counts()
        print(f'Job ID: {job.job_id()}\nJob status: {job.status()}')
        raw_data = list(counts.keys())
        data = ''.join(raw_data)
        
    return data     # Returns a concatenated string of all binary digits in the order that they were measured

In [ ]:
# Increased QRNG on ibm_sherbrooke
def number_generator_sherbrooke(num_qubits, num_shots):
    circ = QuantumCircuit(num_qubits, num_qubits)                 # Creates circuit with number of qubits obtained
    circ.h(range(num_qubits))                                     # Applies a hadamard gate to all qubits
    circ.measure(range(num_qubits), range(num_qubits))            # Measures all qubits and assigns them to classical bits

    # Runs the QRNG on ibm_sherbooke with num_shots
    sherbooke_backend = service.backend('ibm_sherbooke')          # Creates a backend with ibm_sherbooke
    pm = generate_preset_pass_manager(backend=sherbooke_backend, optimization_level=3)   # Can change optimization_level to whatever's desired
    isa_circuit = pm.run(circ)

    with Session(backend=sherbooke_backend) as session:
        sampler = Sampler(mode=session)
        job = sampler.run([isa_circuit], shots=num_shots)
        counts = job.result()[0].data.c.get_counts()
        print(f'Job ID: {job.job_id()}\nJob status: {job.status()}')
        raw_data = list(counts.keys())
        data = ''.join(raw_data)

    return data     # Returns a concatenated string of all binary digits in the order that they were measured

In [ ]:
# Iteration method
# Memory limitations (simulator only 30) -> "chunk" bit input
# chunk_size is # of qubits to divide chunk in to
def iterationChunker(machine, chunk_size):
    chunks = int(np.floor(num_qubits / chunk_size))
    remainder = int(num_qubits - chunks * chunk_size)
    rand_num = ''

    for i in range(0, chunks):
        if machine == 'simulator':
            rand_num += number_generator_simulator(chunk_size, num_shots)
        elif machine == 'ibm_brisbane':
            rand_num += number_generator_brisbane(chunk_size, num_shots)
        elif machine == 'ibm_sherbrooke':
            rand_num += number_generator_sherbrooke(chunk_size, num_shots)

    if remainder != 0:
        for i in range(0, chunks):
            if machine == 'simulator':
                rand_num += number_generator_simulator(chunk_size, num_shots)
            elif machine == 'ibm_brisbane':
                rand_num += number_generator_brisbane(chunk_size, num_shots)
            elif machine == 'ibm_sherbrooke':
                rand_num += number_generator_sherbrooke(chunk_size, num_shots)

    return rand_num

In [ ]:
# Mod2 method
def mod2(machine, chunk_size, mod2_mods):
    iterations = mod2_mods
    outputs = []
    final_rand_num = ''

    for i in range(iterations):
        outputs.append(iterationChunker(machine, chunk_size))

    for i in range(0, iterations, mod2_mods):
        pair_xor = ''
        
        for j in range (0, len(outputs[i])):
            cur_char = 0
            for k in range (0, mod2_mods):
                cur_char += int(outputs[i + k][j])
            pair_xor += str(cur_char % 2)

        final_rand_num += pair_xor
    
    return final_rand_num

In [ ]:
data = mod2(machine, chunk_size, mod2_mods)
if machine == 'simulator':
    with open(f'{machine}_{num_qubits}_{num_shots}_{chunk_size}_{mod2_mods}.txt', 'w') as f:
        for shot in data:
            f.write(shot)
elif machine == 'ibm_brisbane':
    with open(f'{machine}_{num_qubits}_{num_shots}_{chunk_size}_{mod2_mods}.txt', 'w') as f:
        for shot in data:
            f.write(shot)
elif machine == 'ibm_sherbrooke':
    with open(f'{machine}_{num_qubits}_{num_shots}_{chunk_size}_{mod2_mods}.txt', 'w') as f:
        for shot in data:
            f.write(shot)

KeyboardInterrupt: 